In [1]:
!pip install transformers

In [13]:
# We start by importing GT2
from transformers import GPT2Tokenizer, GPT2LMHeadModel

import torch

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# We load the model.
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [4]:
text = "In an old city full of castles and penguins"
# We tokenize the text
tokenized_text = tokenizer.encode(text, return_tensors="pt")

In [5]:
decoded_output = tokenizer.decode(tokenized_text[0], skip_special_tokens=True)
decoded_output

'In an old city full of castles and penguins'

In [10]:
# We generate the output.
output = model.generate(tokenized_text, max_length=15, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=3)

# We decode the output.
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In an old city full of castles and penguins, you've got a


## Greedy search.
- Let's implement the method `generate` by selecting the word with the highest probability at each step.

In [14]:
def generate_greedy_search(model, tokenizer, text, max_length):
    """
    Generates text using greedy search.

    Parameters
    ----------
    model: GPT2LMHeadModel
        The model to use for generating text.
    tokenizer: GPT2Tokenizer
        The tokenizer to use for generating text.
    text: str
        The text to use as a starting point for generating text.
    max_length: int
        The maximum length of the generated text.
    """
    # Tokenize the text.
    tokenized_text = tokenizer.encode(text, return_tensors="pt")

    for _ in range(max_length):
        logits = model(tokenized_text)[0]
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokenized_text = torch.cat([tokenized_text, next_token.unsqueeze(0)], dim=-1)
    # Decode the tokenized text.
    decoded_output = tokenizer.decode(tokenized_text[0], skip_special_tokens=True)
    return decoded_output

## Beam search.

In [23]:
def generate_beam_search(model, tokenizer, text, max_length, beam_size=5):
    """
    Generates text using beam search.

    Parameters
    ----------
    model: GPT2LMHeadModel
        The model to use for generating text.
    tokenizer: GPT2Tokenizer
        The tokenizer to use for generating text.
    text: str
        The text to use as a starting point for generating text.
    max_length: int
        The maximum length of the generated text.
    beam_size: int, default 5
        The beam size for the search.

    Returns
    -------
    list of str
        A list of the top k generated texts with their respective scores.
    """
    # Tokenize the text.
    tokenized_text = tokenizer.encode(text, return_tensors="pt")

    # Initialize the beam search.
    beam = [{"tokens": tokenized_text, "score": 0.0}]

    for _ in range(max_length):
        # Create a list to store the new candidates for the beam.
        candidates = []

        # Generate candidates for each element in the current beam.
        for elem in beam:
            # Get the last token of the current candidate.
            last_token = elem["tokens"][:, -1:]

            # Get the logits for the next token using the model.
            logits = model(elem["tokens"])[0][:, -1, :]

            # Apply a log softmax to the logits.
            log_probs = torch.log_softmax(logits, dim=-1)

            # Calculate the scores of the next tokens by adding the log probabilities to the scores of the current candidate.
            scores = log_probs + elem["score"]

            # Get the top k scores and their corresponding tokens.
            topk_scores, topk_tokens = torch.topk(scores, k=beam_size, dim=-1)

            # Create new candidates for each of the top k tokens.
            for i in range(beam_size):
                candidate = {}
                candidate["tokens"] = torch.cat([elem["tokens"], topk_tokens[:, i:i+1]], dim=-1)
                candidate["score"] = topk_scores[:, i].item()
                candidates.append(candidate)

        # Sort the candidates by score and keep the top k.
        candidates = sorted(candidates, key=lambda x: x["score"], reverse=True)[:beam_size]

        # Check if the top candidate has an end-of-text token.
        top_candidate_tokens = candidates[0]["tokens"]
        if top_candidate_tokens[0, -1] == tokenizer.eos_token_id:
            # If so, return the generated text.
            generated_text = tokenizer.decode(top_candidate_tokens[0], skip_special_tokens=True)
            return generated_text

        # Update the beam with the new candidates.
        beam = candidates

    # Return the top k generated texts.
    generated_texts = [(tokenizer.decode(candidate["tokens"][0], skip_special_tokens=True), candidate["score"]) for candidate in beam]
    return generated_texts
